In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())

【訊息】目前的工作目錄: D:\Mypython\專題


In [2]:
#開啟欄位到最大，方便 pandas 匯入檔案
csv.field_size_limit(int(sys.maxsize/10000000000))

131072

In [41]:
df=pd.read_csv(r'D:\Mypython\專題\dealQQ.csv', sep=',', engine='python')

In [4]:
df.shape

(546688, 44)

In [5]:
df.columns

Index(['business_hour_guid', 'seller_GUID', 'business_hour_order_time_s',
       'business_hour_order_time_e', 'business_hour_deliver_time_s',
       'business_hour_deliver_time_e', 'pic_alt', 'zip_code', 'city_name',
       'district_name', 'item_name', 'item_orig_price', 'item_price',
       'item_default_daily_amount', 'event_name', 'event_title',
       'product_spec', 'shopping_cart', 'combo_pack_count',
       'quantity_multiplier', 'city_list', 'deal_acc_business_group_id',
       'acc_business_group_name', 'deal_type', 'is_travel_deal',
       'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'shippingdate', 'is_long_contract',
       'develope_sales_id', 'operation_sales_id', 'slug', 'ordered_quantity',
       'ordered_total', 'category_list', 'freight_amount', 'enable_isp',
       'discount_price', 'is_wms', 'ancestor_business_hour_guid', 'code_name',
       'code'],
      dtype='object')

In [7]:
df.head(2)

Wall time: 0 ns


,business_hour_guid,seller_GUID,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,pic_alt,zip_code,city_name,district_name,...,ordered_quantity,ordered_total,category_list,freight_amount,enable_isp,discount_price,is_wms,ancestor_business_hour_guid,code_name,code
0,29B37E53-021C-48E9-909E-C6C151974AC7,EC5DDAD4-F8F8-418B-8AED-7BAC2553F2B6,00:00.0,00:00.0,00:00.0,00:59.0,柯絲妃兒/胎盤素/肌因賦活/身體精華乳,407.0,台中市,西屯區,...,0.0,0.0,[88|221|124|769|136|139],0.0,0.0,NaN,0.0,NaN,美體私密,5821.0
1,2C6E83A9-9785-4593-A759-07A78AB309BB,B6CC8A1E-37C8-44E5-B6E2-15123387AF95,00:00.0,00:00.0,00:00.0,00:59.0,aibo 領導者/ 立體聲/智慧藍牙/耳機麥克風系列,243.0,新北市,泰山區,...,0.0,0.0,[88|221|117|2179|2180|100105|100106|136|139],0.0,0.0,NaN,0.0,NaN,網路設備/穿戴裝置/辦公設備,222.0


In [8]:
take_all_word = []
for i in range(0,100):
    take_word = []
    for t in open(r'D:\Mypython\專題\分群{}.txt'.format(i)):
        take_word.append(t.rstrip())
    print(take_word)
    take_all_word.append(take_word)

['電源', '行動', '充電器', '無線', 'usb', '迷你', '充電線', '充電', '認證', '多功能', '手機', '旅行', 'nan', '大容量', 'led', '三合一', '隨身', '一送', '智慧', '智能', '兒童', '蘋果', '金屬', '電池', '支架']
['自動', 'uv', '雨衣', '晴雨傘', '防風', '超大', '黑膠', '收傘', '防曬', '雨鞋', '防雨', '迷你', '防滑', '雙層', '晴雨', '防水', '自動雨傘', '自動傘', '時尚', '日系', '輕量', '反光', '遮陽帽', '經典', '隨身']
['喇叭', '藍芽', '藍牙', '隨身', '投影', '麥克風', '無線', '行動', '電視盒', '重低音', '音箱', '耳機', '音響', '盒子', '神器', '電視', '卡拉', '迷你', '智慧', '多功能', '手機', 'led', '高清', '自拍', '繽紛']
['認證', '頂級', '嚴選', '健康', '益生菌', '幼兒', '牛奶', '軟糖', '進口', '珍珠', '天然', '禮盒', '特級', '百香果', '地瓜', '蘋果', '異果', '3d', '超人', '買新鮮', '櫻桃', '芒果', '番茄', '台斤', '蕃茄']
['無線', '充電線', '電源', 'usb', '支架', '行動', '耳機', '充電器', '迷你', '藍牙耳機', '合金', '充電', '藍芽', '智能', '隨身', '傳輸', '手機', '多功能', '輸線', '防水', '認證', 'led', '神器', '磁吸', '三合一']
['行動', '平板', '電源', '電腦', '皮套', '入組', 'usb', '到貨', '彩色', '大容量', '保護', '核心', '專用', '手機', '隨身', 'ipad', '超薄', 'iphone', '8g', '組合', '手工', '智慧型', '10', '傳輸線', '3c']
['巧克力', '手工', '禮盒', '食品', '海苔', '綜合', '堅果', '杏仁', '超人',

In [9]:
take_all_word[0]

['電源',
 '行動',
 '充電器',
 '無線',
 'usb',
 '迷你',
 '充電線',
 '充電',
 '認證',
 '多功能',
 '手機',
 '旅行',
 'nan',
 '大容量',
 'led',
 '三合一',
 '隨身',
 '一送',
 '智慧',
 '智能',
 '兒童',
 '蘋果',
 '金屬',
 '電池',
 '支架']

In [18]:
def word(x):
    l = list(jieba.cut(x))  # 匯入的資料進行jieba切割
    stroke = []
    for i in range(0,100):
        c = 0
        for j in l:
            if j in take_all_word[i]:
                c = c + 1
        stroke.append(c)
    return(stroke)

In [11]:
df['item_name'] = df['item_name'].astype('str')

### 取10筆

In [23]:
BB = []
BB.append(list(df['item_name'][:10].apply(word)))

In [24]:
BB

[[[0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0],
  [1,
   0,
   2,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   

In [25]:
pd.DataFrame(BB[0])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
pd.DataFrame(BB[0]).to_csv(r"D:\MyPython\專題\答.csv")

### 取全部

In [27]:
AA = []
AA.append(list(df['item_name'].apply(word)))

In [28]:
pd.DataFrame(AA[0])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,1,0,2,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [29]:
pd.DataFrame(AA[0]).to_csv(r"D:\MyPython\專題\答.csv")

### 抽出商品名稱

In [34]:
pd.DataFrame(list(df['item_name'])).to_csv(r"D:\MyPython\專題\品名.csv",encoding='utf-8-sig')

### 抽出對應的Google_ID

In [47]:
df['code'].fillna = 'NULL'

In [55]:
df['code'] = df['code'].astype('str')

In [69]:
code_id = list(df['code'])

In [70]:
def translate(code_id):
    subs = {
        'nan': 'NULL'
        }

In [71]:
translate(code_id)

In [72]:
code_id

['5821.0',
 '222.0',
 '222.0',
 '222.0',
 '222.0',
 '222.0',
 '423.0',
 '423.0',
 '423.0',
 '423.0',
 '423.0',
 '430.0',
 '536.0',
 '432.0',
 '990.0',
 '990.0',
 '990.0',
 '990.0',
 '499969.0',
 '499969.0',
 '990.0',
 '2082.0',
 '990.0',
 '990.0',
 '990.0',
 '499969.0',
 '990.0',
 '990.0',
 '730.0',
 '922.0',
 '2890.0',
 '2890.0',
 '2890.0',
 '630.0',
 '630.0',
 '630.0',
 '630.0',
 '630.0',
 '630.0',
 '1011.0',
 '730.0',
 '730.0',
 '166.0',
 '166.0',
 '166.0',
 '166.0',
 '166.0',
 '536.0',
 '1011.0',
 '1011.0',
 '1011.0',
 '166.0',
 '432.0',
 '536.0',
 '567.0',
 '536.0',
 '536.0',
 '536.0',
 '536.0',
 '1604.0',
 '166.0',
 '536.0',
 '1876.0',
 '1876.0',
 '630.0',
 '1876.0',
 '536.0',
 '536.0',
 '536.0',
 '536.0',
 '2082.0',
 '2082.0',
 '2082.0',
 '2082.0',
 '1604.0',
 '423.0',
 '536.0',
 '1604.0',
 '1604.0',
 '536.0',
 '536.0',
 '536.0',
 '423.0',
 '1604.0',
 '536.0',
 '1604.0',
 '423.0',
 '423.0',
 '1604.0',
 '1604.0',
 '1604.0',
 '1604.0',
 '1604.0',
 '1604.0',
 '1604.0',
 '1604.0',
 

In [74]:
pd.DataFrame(code_id).to_csv(r"D:\MyPython\專題\googleID.csv",encoding='utf-8-sig')